### Can we classify each phase as relatively low or high anxiety for each subject? ###
#### APD, WESAD ####

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE,
# ]
] + train.Metrics.STATISTICAL

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

model_phases_wesad = dr_w.Phases.PHASE_ORDER

anxiety_label_type = None
wesad_label_type = "stai"

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40],
        "eval_metric": ["error", "log_loss"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    "random": None
}

threshold = "fixed"
test_size = 1.0

percent_of_target_dataset = 0.0

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

temp_a, _ = train.Train_WESAD.get_wesad_data([train.Metrics.BPM], phases=dr_w.Phases.PHASE_ORDER, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_wesad_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_wesad_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [3]:
# TRAIN ON APD AND TEST ON WESAD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(38)

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_apd_wesad = {}
ensemble_weights_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_apd_wesad[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models_apd_wesad, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_apd_wesad, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_apd_wesad, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights_apd[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.56725995 0.57611971 0.57885823 0.57901529 0.57501028 0.53131334
 0.56592357 0.56253755 0.57062523 0.56979652 0.54660776 0.55536631
 0.56389933 0.57545636 0.57689887 0.53511752 0.53486643 0.5509702
 0.55359299 0.56312378 0.54053962 0.54338541 0.5606425  0.56688497
 0.56518462        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.52560909 0.52519313 0.52537676 0.52654506 0.53130785 0.52689904
 0.52632962 0.52706271 0.54259215 0.53998568 0.52901871 0.53185727
 0.53818475 0.53669758 0.53856487 0.53802235 0.5405111  0.5524732
 0.55781832 0.56069644 0.53568904 0.54486099 0.54907008 0.55628493
 0.54222897 0.55628541 0.55829861 0.56538033 0.5492737  0.56426384
 0.57353755 0.57411414 0.55805123 0.560793   0.56715815 0.56403483
 0.55517934 0.56781492 0.5637188  0.5648215  0.56663312 0.58362266
 0.58145463 0.57475447 0.55855409 0.575712   0.57465985 0.57260245
 0.55452874 0.56611405 0.57403389 0.57147429 0.5768611  0.56146576
 0.56566591 0.56113176 0.57963666 0.57863299 0.58206353 0.58369988
 0.60586486 0.5906891  0.59225013 0.59094806        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan     

SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'poly', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [428  95], Predictions: [0 1], [ 34 489]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [428  95], Predictions: [0 1], [166 357]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [428  95], Predictions: [0 1], [140 383]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'

In [4]:
# TRAIN ON WESAD AND TEST ON APD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(38)

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
if anxiety_label_type is not None:
    x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_wesad = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_wesad / n_apd)) * n_wesad)
# print(n_wesad / n_apd)
# augment_indices = random.sample(range(n_wesad), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_wesad_apd = {}
ensemble_weights_wesad = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_wesad_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models_wesad_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights_wesad[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.88579448 0.92358315 0.92744062 0.9251813  0.92352759 0.87856108
 0.92626902 0.93176573 0.93116613 0.93234905 0.89495626 0.92421861
 0.93516576 0.93203237 0.93113482 0.90898898 0.92869528 0.93645851
 0.92805175 0.92605802 0.90481599 0.93188252 0.93034399 0.92815869
 0.92513853        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.85850868 0.88491833 0.8900102  0.89499991 0.87608201 0.88262958
 0.88605267 0.88547565 0.87608201 0.88262958 0.88687572 0.88743038
 0.87608201 0.88262958 0.8869786  0.88743038 0.88485859 0.89832918
 0.92095088 0.92860875 0.88236847 0.88307489 0.91463141 0.92507631
 0.88460159 0.90294794 0.92703845 0.93011844 0.88525182 0.89557811
 0.92781327 0.92708668 0.93215447 0.93671808 0.93889686 0.94027806
 0.92411673 0.93105975 0.93667531 0.93843728 0.92536516 0.93082505
 0.93797519 0.94154722 0.92080663 0.92822529 0.93533786 0.93942429
 0.93175251 0.93350209 0.93184247 0.93144104 0.92704894 0.94322333
 0.94127111 0.93999181 0.92779179 0.94326411 0.94177164 0.93918383
 0.92793224 0.94326411 0.94163119 0.93904338        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan    

SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 5, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 6, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [570 462], Predictions: [0 1], [440 592]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [570 462], Predictions: [0 1], [657 375]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [570 462], Predictions: [0 1], [720 312]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 'e

In [5]:
# TRAIN ON WESAD AND TEST ON APD -- SPEECH TASK ONLY
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(38)

model_phases_apd_speech = [
    "Baseline_Rest", 
    # "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd_speech, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
if anxiety_label_type is not None:
    x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_wesad = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_wesad / n_apd)) * n_wesad)
# augment_indices = random.sample(range(n_wesad), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_wesad_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_wesad_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models_wesad_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.89482515 0.91996043 0.92799701 0.93903819 0.93767967 0.90095526
 0.91904142 0.92862701 0.93987505 0.93776621 0.90427677 0.91843899
 0.92900861 0.93571217 0.93616415 0.90071892 0.92218517 0.93092175
 0.93227335 0.93286682 0.91379361 0.92623258 0.93137951 0.93658543
 0.93366636        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.8816292  0.88764611 0.89742338 0.89573075 0.88324795 0.88858337
 0.89719731 0.89775911 0.88324795 0.88858337 0.8991873  0.89787181
 0.88324795 0.88858337 0.90045134 0.89843361 0.89877774 0.89145619
 0.93381763 0.93344577 0.90172059 0.91666497 0.93842368 0.94011162
 0.9028124  0.92336815 0.93925572 0.93832251 0.90291528 0.92336815
 0.93982478 0.94115494 0.93458571 0.93814012 0.94250517 0.94252208
 0.93523319 0.94378242 0.94275904 0.94376242 0.94614374 0.95440106
 0.94974048 0.95276141 0.94696679 0.95491546 0.94830015 0.95132108
 0.94082352 0.9534495  0.94821502 0.94796213 0.93537639 0.94718931
 0.94963857 0.94931654 0.93817917 0.9485346  0.95101243 0.94944893
 0.93848781 0.9485346  0.95111531 0.94955181        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan    

SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'binary_logloss', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [341 243], Predictions: [0 1], [269 315]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [341 243], Predictions: [0 1], [385 199]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [341 243], Predictions: [0 1], [385 199]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 'e

In [6]:
# TRAIN ON WESAD AND TEST ON APD -- BUG TASK ONLY
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(38)

model_phases_apd_speech = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    # "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd_speech, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
if anxiety_label_type is not None:
    x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_wesad = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_wesad / n_apd)) * n_wesad)
# augment_indices = random.sample(range(n_wesad), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_wesad_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_wesad_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models_wesad_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.92473276 0.93514677 0.9454559  0.94937484 0.94850263 0.94077242
 0.94635073 0.94571818 0.94862131 0.94964426 0.93989188 0.95225135
 0.95034174 0.95062034 0.95138669 0.93610021 0.94576285 0.94623827
 0.95042892 0.95553085 0.93385372 0.94527872 0.96079024 0.96141146
 0.96018399        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.86613744 0.8640074  0.86448587 0.87357386 0.87037117 0.86800453
 0.87152322 0.90176603 0.88280094 0.88284698 0.87991308 0.90893963
 0.88223914 0.88284698 0.87991308 0.90879918 0.88789646 0.92437135
 0.94630808 0.94941893 0.9076897  0.94130179 0.95912579 0.96333044
 0.91586661 0.92917594 0.94301055 0.9617178  0.91710311 0.92888253
 0.94848114 0.96476413 0.9446289  0.954071   0.94686653 0.94216953
 0.96310882 0.95630675 0.95496324 0.9523673  0.92105746 0.9554875
 0.9509562  0.94958081 0.93990543 0.96030752 0.96238288 0.95999209
 0.95827212 0.95122618 0.94599099 0.94231631 0.95222743 0.94223587
 0.94128879 0.93846413 0.94837984 0.94323868 0.93908446 0.93777421
 0.95493848 0.94731171 0.94410308 0.94040518        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan     

SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [297 255], Predictions: [0 1], [227 325]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [297 255], Predictions: [0 1], [380 172]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [297 255], Predictions: [0 1], [372 180]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 'e

In [7]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier


random.seed(38)

percent_of_target_dataset = 0.0
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]


def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


In [8]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)

from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


random.seed(38)

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

voting_type = "weighted_avg"
# voting_type = "majority_vote"

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

subjects = list(np.unique(x_b.loc[:, "subject"]))
train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
x_train = pd.concat([x_a, x_train_addition])
y_train = pd.concat([y_a, y_train_addition])
x_test = x_b[~x_b["subject"].isin(train_subjects)]
y_test = y_b[~y_b["subject"].isin(train_subjects)]

y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models_apd_wesad[name] for name in ensemble_models}
weights = [ensemble_weights_apd[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
# weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)

print("Train APD, test WESAD")
print(f"APD size: {x_train.shape[0]}\n{y_train.value_counts()}\nWESAD size: {x_test.shape[0]}\n{y_test.value_counts()}")
print("Predictions:")
print(f"{np.array(np.unique(y_pred, return_counts=True)).T}")
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

###############################################################################################################

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_wesad = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_wesad / n_apd)) * n_wesad)
# augment_indices = random.sample(range(n_wesad), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

subjects = list(np.unique(x_b.loc[:, "subject"]))
train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
x_train = pd.concat([x_a, x_train_addition])
y_train = pd.concat([y_a, y_train_addition])
x_test = x_b[~x_b["subject"].isin(train_subjects)]
y_test = y_b[~y_b["subject"].isin(train_subjects)]

y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models_wesad_apd[name] for name in ensemble_models}
weights = [ensemble_weights_wesad[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
# weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print("Train WESAD, test APD")
print(f"WESAD size: {x_train.shape[0]}\n{y_train.value_counts()}\nAPD size: {x_test.shape[0]}\n{y_test.value_counts()}")
print("Predictions:")
print(f"{np.array(np.unique(y_pred, return_counts=True)).T}")
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

Train APD, test WESAD
APD size: 1032
0    570
1    462
Name: label, dtype: int64
WESAD size: 523
0    428
1     95
Name: label, dtype: int64
Predictions:
[[  0 239]
 [  1 284]]
	Accuracy: 0.4780114722753346
	Precision: 0.18661971830985916
	Recall: 0.5578947368421052
	F1-score: 0.2796833773087071
	AUC score: 0.5090875553369405
----------------------------------------
Train WESAD, test APD
WESAD size: 523
0    428
1     95
Name: label, dtype: int64
APD size: 1032
0    570
1    462
Name: label, dtype: int64
Predictions:
[[  0 696]
 [  1 336]]
	Accuracy: 0.5678294573643411
	Precision: 0.5238095238095238
	Recall: 0.38095238095238093
	F1-score: 0.4411027568922306
	AUC score: 0.550125313283208
----------------------------------------
